## Feature Engineering and CV based Winners' Solutions

continued from eda_4_26

New in this notebook:
- average of products for each (customer, product) pair
- exponent weighted average of products each (customer, product) pair
- time since presence of products, distance to the first 1
- time to the last positive flank (01)
- time to the last negative flank (10)
- time to the last 1, to the nearest product purchase
- time to the first 1, to the first product purchase

Trained@2015-06-28, validated@2015-12-28, mlogloss=1.28481

Private score: 0.0302054, public score: 0.0298683

To-do: 
- mean encoding of products grouped by combinations of: canal_entrada, segmento, cod_prov
- Time since change and lags for a few non-product features: 
    - segmento
    - ind_actividad_cliente
    - cod_prov
    - canal_entrada
    - indrel_1mes
    - tiprel_1mes


Features:
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

In [1]:
from santander_helper import *

In [6]:
def cv_xgb_skfrm(params, x_train, y_train, num_boost_round=3, n_splits=3, 
                           n_repeats=2, random_state=0, verbose_eval=False):
    '''
    CV of xgb using Stratified KFold Repeated Models (SKFRM)
    verbose_eval is the same as in xgb.train
    '''
    cv_results = {}
    clfs = {}
    running_time = {}
    
    eval_metric = params['eval_metric']
    
    np.random.seed(random_state)
    skf = StratifiedKFold(n_splits=n_splits, random_state=np.random.randint(10**6), shuffle=True)
    
    for m in range(n_repeats):
        for n, (train_index, val_index) in enumerate(skf.split(x_train, y_train)):
            
            start_time = time.time()
            
            # Construct DMatrix
            dtrain = xgb.DMatrix(x_train.iloc[train_index], label=y_train.iloc[train_index])
            dval = xgb.DMatrix(x_train.iloc[val_index], label=y_train.iloc[val_index])
            
            # Placeholder for evals_result
            cv_results[m, n] = {}
            params['seed'] = np.random.randint(10**6)
            clfs[m, n] = xgb.train(params, dtrain, num_boost_round=num_boost_round,
                                   evals=[(dtrain, 'train'), (dval, 'val')], 
                                   maximize=True, early_stopping_rounds=None, 
                                   evals_result=cv_results[m, n], verbose_eval=verbose_eval)
        
            running_time[m, n] = time.time() - start_time
            
            print('Repeat {}, split {}, validate score = {:.3f}, running time = {:.3f} min'.format(m, n, 
                cv_results[m, n]['val'][eval_metric][-1], running_time[m, n]/60))
        
    # Post-process cv_results
    cv_results_final = {}
    for m in range(n_repeats):
        for n in range(n_splits):
            cv_results_final['train', m, n] = cv_results[m, n]['train'][eval_metric]
            cv_results_final['val', m, n] = cv_results[m, n]['val'][eval_metric]
    
    df = pd.DataFrame.from_dict(cv_results_final)
    df.index.name = 'iteration'
    df.columns.names = ['dataset', 'repeat', 'split']

    print('Score mean = {:.3f}, std = {:.3f}'.format(df['val'].iloc[-1].mean(), df['val'].iloc[-1].std()))
    
    return df, clfs, running_time


In [ ]:
x_train_june15, y_train_june15 = create_train('2015-06-28', pattern_flag=True)

params = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

df, clfs, running_time = cv_xgb_skfrm(params, x_train_june15, y_train_june15, 
                                      verbose_eval=False, num_boost_round=100, 
                                      n_splits=4, n_repeats=8)

Repeat 0, split 0, validate score = 0.995, running time = 2.352 min
Repeat 0, split 1, validate score = 0.999, running time = 2.313 min
Repeat 0, split 2, validate score = 0.997, running time = 2.306 min
Repeat 0, split 3, validate score = 0.998, running time = 2.321 min
Repeat 1, split 0, validate score = 0.996, running time = 2.384 min
Repeat 1, split 1, validate score = 0.999, running time = 2.300 min
Repeat 1, split 2, validate score = 0.999, running time = 2.336 min
Repeat 1, split 3, validate score = 0.997, running time = 2.344 min


In [11]:
df.mean(axis=1, level=0)

dataset,train,val
iteration,,
0,2.705467,2.711566
1,2.531360,2.542907
2,2.392536,2.409143


### 2015-06-28, max_lag=5

In [7]:
x_train_june15, y_train_june15 = create_train('2015-06-28', pattern_flag=True)
x_val_june15, y_val_june15 = create_train('2016-05-28', pattern_flag=True)

x_test_june15 = create_test(pattern_flag=True)

param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}
num_rounds = 100

dtrain_june15 = xgb.DMatrix(x_train_june15.values, y_train_june15.values)
dval_june15 = xgb.DMatrix(x_val_june15.values, y_val_june15.values)
model_june15 = xgb.train(param, dtrain_june15, num_rounds, evals=[(dtrain_june15, 'train'), (dval_june15, 'dval')], verbose_eval=True)

preds_june15 = model_june15.predict(xgb.DMatrix(x_test_june15.values))

df_preds_june15 = pd.DataFrame(preds_june15, index=x_test_june15.index, columns=target_cols)
# Remove already bought products 
df_preds_june15[x_test_june15[target_cols]==1] = 0 
preds_june15 = df_preds_june15.values
preds_june15 = np.argsort(preds_june15, axis=1)
preds_june15 = np.fliplr(preds_june15)[:, :7]

test_id = x_test_june15.loc[:, 'ncodpers'].values
final_preds_june15 = [' '.join([target_cols[k] for k in pred]) for pred in preds_june15]

out_df_june15 = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds_june15})
out_df_june15.to_csv('eda_4_28_june15.csv.gz', compression='gzip', index=False)

[0]	train-mlogloss:2.70361	dval-mlogloss:2.73293
[1]	train-mlogloss:2.53061	dval-mlogloss:2.57631
[2]	train-mlogloss:2.39202	dval-mlogloss:2.45619
[3]	train-mlogloss:2.27645	dval-mlogloss:2.35394
[4]	train-mlogloss:2.1773	dval-mlogloss:2.26842
[5]	train-mlogloss:2.09068	dval-mlogloss:2.18941
[6]	train-mlogloss:2.01344	dval-mlogloss:2.12208
[7]	train-mlogloss:1.94366	dval-mlogloss:2.06529
[8]	train-mlogloss:1.88058	dval-mlogloss:2.00993
[9]	train-mlogloss:1.82338	dval-mlogloss:1.95961
[10]	train-mlogloss:1.77048	dval-mlogloss:1.91311
[11]	train-mlogloss:1.72147	dval-mlogloss:1.86929
[12]	train-mlogloss:1.67601	dval-mlogloss:1.83026
[13]	train-mlogloss:1.63373	dval-mlogloss:1.79266
[14]	train-mlogloss:1.59426	dval-mlogloss:1.75835
[15]	train-mlogloss:1.55716	dval-mlogloss:1.72968
[16]	train-mlogloss:1.52252	dval-mlogloss:1.6984
[17]	train-mlogloss:1.49021	dval-mlogloss:1.67133
[18]	train-mlogloss:1.45926	dval-mlogloss:1.64782
[19]	train-mlogloss:1.43075	dval-mlogloss:1.62538
[20]	train-m

KeyboardInterrupt: 

### 2016-05-28, max_lag=16

In [3]:
x_train_may16, y_train_may16 = create_train('2015-05-28', pattern_flag=True, max_lag=16)

x_test_may16 = create_test(pattern_flag=True)

param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}
num_rounds = 60

dtrain_may16 = xgb.DMatrix(x_train_may16.values, y_train_may16.values)
model_may16 = xgb.train(param, dtrain_may16, num_rounds, evals=[(dtrain_may16, 'train')], verbose_eval=True)

preds_may16 = model_may16.predict(xgb.DMatrix(x_test_may16.values))

df_preds_may16 = pd.DataFrame(preds_may16, index=x_test_may16.index, columns=target_cols)
# Remove already bought products 
df_preds_may16[x_test_may16[target_cols]==1] = 0 
preds_may16 = df_preds_may16.values
preds_may16 = np.argsort(preds_may16, axis=1)
preds_may16 = np.fliplr(preds_may16)[:, :7]

test_id = x_test_may16.loc[:, 'ncodpers'].values
final_preds_may16 = [' '.join([target_cols[k] for k in pred]) for pred in preds_may16]

out_df_may16 = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds_may16})
out_df_may16.to_csv('eda_4_28_may16.csv.gz', compression='gzip', index=False)

[0]	train-mlogloss:2.64321
[1]	train-mlogloss:2.42916
[2]	train-mlogloss:2.26143
[3]	train-mlogloss:2.12057
[4]	train-mlogloss:2.00099
[5]	train-mlogloss:1.89603
[6]	train-mlogloss:1.80356
[7]	train-mlogloss:1.72067
[8]	train-mlogloss:1.64465
[9]	train-mlogloss:1.57663
[10]	train-mlogloss:1.51355
[11]	train-mlogloss:1.45557
[12]	train-mlogloss:1.40214
[13]	train-mlogloss:1.35231
[14]	train-mlogloss:1.30587
[15]	train-mlogloss:1.26249
[16]	train-mlogloss:1.222
[17]	train-mlogloss:1.18383
[18]	train-mlogloss:1.14811
[19]	train-mlogloss:1.11448
[20]	train-mlogloss:1.08268
[21]	train-mlogloss:1.0525
[22]	train-mlogloss:1.02404
[23]	train-mlogloss:0.997108
[24]	train-mlogloss:0.971606
[25]	train-mlogloss:0.947348
[26]	train-mlogloss:0.924398
[27]	train-mlogloss:0.902768
[28]	train-mlogloss:0.881792
[29]	train-mlogloss:0.861978
[30]	train-mlogloss:0.842969
[31]	train-mlogloss:0.824979
[32]	train-mlogloss:0.80799
[33]	train-mlogloss:0.791558
[34]	train-mlogloss:0.776193
[35]	train-mlogloss:0.

### Combine two models

In [4]:
preds_june15 = model_june15.predict(xgb.DMatrix(x_test_june15.values))
preds_may16 = model_may16.predict(xgb.DMatrix(x_test_may16.values))

preds1 = np.sqrt(preds_june15*preds_may16)
preds2 = 0.5*preds_june15 + 0.5*preds_may16

# Geometry mean
df_preds1 = pd.DataFrame(preds1, index=x_test_may16.index, columns=target_cols)
# Remove already bought products 
df_preds1[x_test_may16[target_cols]==1] = 0 
preds1 = df_preds1.values
preds1 = np.argsort(preds1, axis=1)
preds1 = np.fliplr(preds1)[:, :7]

test_id = x_test_may16.loc[:, 'ncodpers'].values
final_preds1 = [' '.join([target_cols[k] for k in pred]) for pred in preds1]

out_df1 = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds1})
out_df1.to_csv('eda_4_28_gm.csv.gz', compression='gzip', index=False)

# Algorithmic mean
df_preds2 = pd.DataFrame(preds2, index=x_test_may16.index, columns=target_cols)
# Remove already bought products 
df_preds2[x_test_may16[target_cols]==1] = 0 
preds2 = df_preds2.values
preds2 = np.argsort(preds2, axis=1)
preds2 = np.fliplr(preds2)[:, :7]

test_id = x_test_may16.loc[:, 'ncodpers'].values
final_preds2 = [' '.join([target_cols[k] for k in pred]) for pred in preds2]

out_df2 = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds2})
out_df2.to_csv('eda_4_28_am.csv.gz', compression='gzip', index=False)
